# **Distribución % de créditos por organismo**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

## 2022

In [2]:
finance = df_financiamientos_2022_puebla[['id','organismo']]
finance

,id,organismo
0,1,2
1,2,2
2,3,2
3,4,2
4,5,1
...,...,...
3099,3100,1
3100,3101,1
3101,3102,1
3102,3103,1


In [3]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)


# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -      45
1  BANJERCITO      15
2        CNBV    1220
3      CONAVI      50
4    FOVISSTE     364
5   INFONAVIT    1391
6       INSUS      19


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1672\346408341.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2' '2' '2' ... '1' '1' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'organismo'] = finance['organismo'].astype(str)


In [4]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,1391,0.448131
1,CNBV,1220,0.393041
2,FOVISSTE,364,0.117268
3,CONAVI,50,0.016108
4,-,45,0.014497
5,INSUS,19,0.006121
6,BANJERCITO,15,0.004832


In [5]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  6208  registros en la base
Hay  3104  registros al contar


In [6]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1672\2284548040.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)


,organismo,conteo,promedio
0,INFONAVIT,1391,0.448131
1,CNBV,1220,0.393041
2,FOVISSTE,364,0.117268
6,BANJERCITO,15,0.004832
3,CONAVI,50,0.016108
5,INSUS,19,0.006121


In [7]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')
fig.show()

## 2023

In [14]:
finance = df_financiamientos_2023_puebla[['id','organismo']]
finance

,id,organismo
0,1,4
1,2,1
2,3,1
3,4,2
4,5,2
...,...,...
10832,10833,1
10833,10834,4
10834,10835,4
10835,10836,4


In [15]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)


# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -     552
1  BANJERCITO      42
2        CNBV    3275
3      CONAVI     230
4    FOVISSTE     985
5   INFONAVIT    5654
6       INSUS      99


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1672\346408341.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4' '1' '1' ... '4' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [16]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,5654,0.521731
1,CNBV,3275,0.302205
2,FOVISSTE,985,0.090892
3,-,552,0.050937
4,CONAVI,230,0.021224
5,INSUS,99,0.009135
6,BANJERCITO,42,0.003876


In [17]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  21674  registros en la base
Hay  10837  registros al contar


In [18]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1672\2284548040.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
0,INFONAVIT,5654,0.521731
1,CNBV,3275,0.302205
2,FOVISSTE,985,0.090892
6,BANJERCITO,42,0.003876
4,CONAVI,230,0.021224
5,INSUS,99,0.009135


In [19]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2023', carpeta='assets\graficas')
fig.show()

## 2024

In [8]:
finance = df_financiamientos_2024_puebla[['id','organismo']]
finance

,id,organismo
0,1,4
1,2,2
2,3,4
3,4,4
4,5,4
...,...,...
2492,2493,4
2493,2494,4
2494,2495,4
2495,2496,4


In [9]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)


# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0           -     218
1  BANJERCITO       6
2        CNBV     660
3      CONAVI      18
4    FOVISSTE     187
5   INFONAVIT    1408
6       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1672\346408341.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4' '2' '4' ... '4' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [10]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = df_conteo_organismos['conteo'] / total_registros
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,1408,0.563877
1,CNBV,660,0.264317
2,-,218,0.087305
3,FOVISSTE,187,0.074890
4,CONAVI,18,0.007209
5,BANJERCITO,6,0.002403


In [11]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  4994  registros en la base
Hay  2497  registros al contar


In [12]:
df_conteo_organismos = df_conteo_organismos[df_conteo_organismos['organismo'] != '-']

# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1672\2284548040.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,organismo,conteo,promedio
0,INFONAVIT,1408,0.563877
1,CNBV,660,0.264317
3,FOVISSTE,187,0.074890
5,BANJERCITO,6,0.002403
4,CONAVI,18,0.007209


In [13]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2024', carpeta='assets\graficas')
fig.show()